In [1]:
from torch_geometric.utils import to_networkx
import networkx as nx
import torch

def find_all_rings(graph_data):

    nx_graph = to_networkx(graph_data, to_undirected=True)
    cycles = list(nx.simple_cycles(nx_graph))

    return cycles

def have_edge(adj, node1, node2):

    return adj[node1, node2] == 1 or adj[node2, node1] == 1


def get_S(data):
    S=torch.zeros(max_nodes,max_nodes)
    adj=torch.zeros((data.num_nodes, data.num_nodes))
    adj[data.edge_index[0], data.edge_index[1]] = 1
    rings=find_all_rings(data)
    
    if len(rings)==0:
        return torch.eye(max_nodes)
    i=0
    for ring in rings:
        if i>=max_nodes:
            return torch.eye(max_nodes)
        if len(ring)<9:
            for j in ring:
                S[j][i]=1.0
            i+=1
            
    for j in range(data.x.shape[0]):
        if i>=max_nodes:
            return torch.eye(max_nodes)
        if S[j,:].max()==0:
            S[j,i]=1.0
            for k in range(j+1,data.num_nodes):
                if S[k,:].max()==0 and have_edge(adj, j, k):
                    S[k,i]=1.0
            i+=1
            
    return S



import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch_geometric.utils import from_smiles

def split_list(input_list, ratio):
    input_list_copy = input_list.copy()
    total_length = len(input_list_copy)
    a_length = int(ratio[0] * total_length)
    b_length = int(ratio[1] * total_length)
    
    random.shuffle(input_list_copy)
    part_a = input_list_copy[:a_length]
    part_b = input_list_copy[a_length:a_length + b_length]
    part_c = input_list_copy[a_length + b_length:]

    return part_a, part_b, part_c

def load_dataset(filename,x_name,y_name1,y_name2):
    table=pd.read_csv(filename)
    result=[]
    for i in tqdm(range(table.shape[0])):
        cur_data=from_smiles(table[x_name][i])
        cur_data.y=torch.tensor([[table[y_name1][i],table[y_name2][i]]],dtype=torch.float32)
        cur_data.S=get_S(cur_data)
        result.append(cur_data)
    return result

def load_splited_dataset(filename,x_name,y_name1,y_name2,ratio=(0.8,0.1,0.1)):
    all_list=load_dataset(filename,x_name,y_name1,y_name1)
    return split_list(all_list,ratio)

In [2]:
import torch
from torch.nn import Linear, ReLU
from torch_geometric.nn import DenseGCNConv, global_mean_pool,dense_diff_pool

class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,normalize=False,lin=True):
        super(GNN, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.convs.append(DenseGCNConv(in_channels, hidden_channels,normalize))
        self.convs.append(DenseGCNConv(hidden_channels, out_channels,normalize))
        self.act=ReLU()

    def forward(self, x, adj, mask=None):
        
        for step in range(len(self.convs)):
            x = self.act(self.convs[step](x, adj))
        
        return x
class RingPool(torch.nn.Module):
    def __init__(self,in_channels, hidden_channels=32, out_channels=2):
        super(RingPool, self).__init__()
        self.gnn1=GNN(in_channels, hidden_channels, hidden_channels)
        self.gnn2=GNN(hidden_channels, hidden_channels, hidden_channels)
        self.lin1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, out_channels)
        self.act=ReLU()
        
    def forward(self, x, adj, s, mask=None):
        x = self.gnn1(x,adj, mask)
        x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)
        x = self.gnn2(x,adj, mask)
        
        x = x.sum(dim=1)
        x = self.act(self.lin1(x))
        x = self.lin2(x)
        return x
    


In [3]:
from torch_geometric.utils import to_dense_batch,to_dense_adj

def train(model, loader, optimizer, loss_func):

    model.train()
    mae=0
    for data in loader:
        data.x=to_dense_batch(data.x,batch=data.batch,max_num_nodes=max_nodes)[0].float()
        data.adj=to_dense_adj(data.edge_index,batch=data.batch,max_num_nodes=max_nodes)
        data.S=data.S.reshape(-1,max_nodes,max_nodes)
        data=data.cuda()
        optimizer.zero_grad()
        pred=model(data.x, data.adj,data.S)
        mae+=torch.sum(torch.abs(pred-data.y)).item()/2
        loss=loss_func(pred,data.y)
        loss.backward()
        optimizer.step()

    return model,mae/len(loader.dataset)

def test(model, loader):
    model.eval()
    mae=0
    with torch.no_grad():
        for data in loader:
            data.x=to_dense_batch(data.x,batch=data.batch,max_num_nodes=max_nodes)[0].float()
            data.adj=to_dense_adj(data.edge_index,batch=data.batch,max_num_nodes=max_nodes)
            data.S=data.S.reshape(-1,max_nodes,max_nodes)
            data=data.cuda()
            out = model(data.x,data.adj,data.S)  
            mae+=torch.sum(torch.abs(out-data.y)).item()/2
        
    return mae / len(loader.dataset)

# QM9

In [4]:
from torch_geometric.seed import seed_everything

max_nodes=9
QM9=load_dataset("./dataset/QM9/QM9.csv","smiles","homo","lumo")

100%|█████████████████████████████████████████████████████████████████████████| 133247/133247 [03:14<00:00, 685.08it/s]


### Exp 1

In [5]:
seed_everything(1222)
QM9_train,QM9_val,QM9_test=split_list(QM9,(0.8,0.1,0.1))

In [6]:
from torch_geometric.loader import DenseDataLoader,DataLoader


train_loader=DataLoader(QM9_train,batch_size=128, shuffle=True)
val_loader = DataLoader(QM9_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(QM9_test, batch_size=1024, shuffle=False)
num_features=QM9_train[0].num_node_features

In [7]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.022676    Val MAE:0.012551    Test MAE:0.012404     Time:12.421992 s
Epoch 2
Train MAE:0.012059    Val MAE:0.012096    Test MAE:0.012003     Time:12.429872 s
Epoch 3
Train MAE:0.011581    Val MAE:0.011118    Test MAE:0.010988     Time:12.592970 s
Epoch 4
Train MAE:0.011243    Val MAE:0.010850    Test MAE:0.010730     Time:13.256961 s
Epoch 5
Train MAE:0.011182    Val MAE:0.010636    Test MAE:0.010501     Time:12.790881 s
Epoch 6
Train MAE:0.011000    Val MAE:0.010561    Test MAE:0.010454     Time:12.809435 s
Epoch 7
Train MAE:0.010993    Val MAE:0.010177    Test MAE:0.010071     Time:12.725467 s
Epoch 8
Train MAE:0.010686    Val MAE:0.010365    Test MAE:0.010202     Time:12.728839 s
Epoch 9
Train MAE:0.010590    Val MAE:0.010144    Test MAE:0.010072     Time:13.023960 s
Epoch 10
Train MAE:0.010398    Val MAE:0.010151    Test MAE:0.010084     Time:13.019576 s
Epoch 11
Train MAE:0.010225    Val MAE:0.010221    Test MAE:0.010119     Time:12.918438 s
Epoch 12
Train MAE:

In [8]:
epochs=50
train_loader=DataLoader(QM9_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.008017    Val MAE:0.008570    Test MAE:0.008549     Time:27.972265 s
Epoch 2
Train MAE:0.008023    Val MAE:0.008130    Test MAE:0.008107     Time:28.856439 s
Epoch 3
Train MAE:0.008003    Val MAE:0.009079    Test MAE:0.009033     Time:28.098326 s
Epoch 4
Train MAE:0.007949    Val MAE:0.007875    Test MAE:0.007828     Time:27.655796 s
Epoch 5
Train MAE:0.007917    Val MAE:0.007817    Test MAE:0.007761     Time:27.259597 s
Epoch 6
Train MAE:0.007898    Val MAE:0.008894    Test MAE:0.008882     Time:28.234588 s
Epoch 7
Train MAE:0.007906    Val MAE:0.008173    Test MAE:0.008117     Time:28.222977 s
Epoch 8
Train MAE:0.007894    Val MAE:0.007999    Test MAE:0.007976     Time:28.026615 s
Epoch 9
Train MAE:0.007861    Val MAE:0.007865    Test MAE:0.007767     Time:27.388793 s
Epoch 10
Train MAE:0.007828    Val MAE:0.007617    Test MAE:0.007574     Time:30.295032 s
Epoch 11
Train MAE:0.007796    Val MAE:0.007563    Test MAE:0.007480     Time:28.219040 s
Epoch 12
Train MAE:

In [9]:
epochs=20
train_loader=DataLoader(QM9_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.007259    Val MAE:0.007466    Test MAE:0.007490     Time:26.958994 s
Epoch 2
Train MAE:0.007228    Val MAE:0.007197    Test MAE:0.007150     Time:27.044929 s
Epoch 3
Train MAE:0.007234    Val MAE:0.007454    Test MAE:0.007410     Time:26.726254 s
Epoch 4
Train MAE:0.007242    Val MAE:0.007807    Test MAE:0.007845     Time:27.328829 s
Epoch 5
Train MAE:0.007211    Val MAE:0.007324    Test MAE:0.007267     Time:26.963239 s
Epoch 6
Train MAE:0.007199    Val MAE:0.007242    Test MAE:0.007220     Time:27.286200 s
Epoch 7
Train MAE:0.007210    Val MAE:0.007119    Test MAE:0.007072     Time:28.592698 s
Epoch 8
Train MAE:0.007183    Val MAE:0.007348    Test MAE:0.007294     Time:28.891621 s
Epoch 9
Train MAE:0.007168    Val MAE:0.007254    Test MAE:0.007286     Time:27.838886 s
Epoch 10
Train MAE:0.007189    Val MAE:0.007210    Test MAE:0.007203     Time:27.130000 s
Epoch 11
Train MAE:0.007159    Val MAE:0.007093    Test MAE:0.007049     Time:27.515774 s
Epoch 12
Train MAE:

In [10]:
with open("./result/RING_QM9_1.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))

### Exp 2

In [7]:
seed_everything(324)
QM9_train,QM9_val,QM9_test=split_list(QM9,(0.8,0.1,0.1))

In [8]:
from torch_geometric.loader import DenseDataLoader,DataLoader


train_loader=DataLoader(QM9_train,batch_size=128, shuffle=True)
val_loader = DataLoader(QM9_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(QM9_test, batch_size=1024, shuffle=False)
num_features=QM9_train[0].num_node_features

In [9]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.023151    Val MAE:0.017532    Test MAE:0.017392     Time:11.224392 s
Epoch 2
Train MAE:0.016792    Val MAE:0.015906    Test MAE:0.015761     Time:11.818900 s
Epoch 3
Train MAE:0.015789    Val MAE:0.015868    Test MAE:0.015757     Time:12.189643 s
Epoch 4
Train MAE:0.014977    Val MAE:0.015604    Test MAE:0.015476     Time:12.216589 s
Epoch 5
Train MAE:0.014475    Val MAE:0.014381    Test MAE:0.014230     Time:12.402188 s
Epoch 6
Train MAE:0.013925    Val MAE:0.013114    Test MAE:0.012970     Time:12.417152 s
Epoch 7
Train MAE:0.013589    Val MAE:0.013180    Test MAE:0.013082     Time:12.507293 s
Epoch 8
Train MAE:0.013405    Val MAE:0.012466    Test MAE:0.012282     Time:12.391737 s
Epoch 9
Train MAE:0.013052    Val MAE:0.014259    Test MAE:0.014068     Time:12.521020 s
Epoch 10
Train MAE:0.013084    Val MAE:0.012509    Test MAE:0.012351     Time:12.574980 s
Epoch 11
Train MAE:0.012587    Val MAE:0.011902    Test MAE:0.011653     Time:12.750822 s
Epoch 12
Train MAE:

In [10]:
epochs=50
train_loader=DataLoader(QM9_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.009409    Val MAE:0.009275    Test MAE:0.009172     Time:25.678294 s
Epoch 2
Train MAE:0.009380    Val MAE:0.009325    Test MAE:0.009250     Time:26.401731 s
Epoch 3
Train MAE:0.009386    Val MAE:0.010944    Test MAE:0.010753     Time:27.155202 s
Epoch 4
Train MAE:0.009274    Val MAE:0.009280    Test MAE:0.009186     Time:27.578329 s
Epoch 5
Train MAE:0.009273    Val MAE:0.009052    Test MAE:0.008895     Time:28.733521 s
Epoch 6
Train MAE:0.009282    Val MAE:0.008937    Test MAE:0.008778     Time:28.185193 s
Epoch 7
Train MAE:0.009228    Val MAE:0.008972    Test MAE:0.008905     Time:27.770976 s
Epoch 8
Train MAE:0.009174    Val MAE:0.011172    Test MAE:0.010995     Time:28.375756 s
Epoch 9
Train MAE:0.009121    Val MAE:0.009813    Test MAE:0.009592     Time:27.985494 s
Epoch 10
Train MAE:0.009074    Val MAE:0.009815    Test MAE:0.009643     Time:27.984039 s
Epoch 11
Train MAE:0.009067    Val MAE:0.008733    Test MAE:0.008664     Time:29.149684 s
Epoch 12
Train MAE:

In [11]:
with open("./result/RING_QM9_2.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))

### Exp 3

In [12]:
seed_everything(10086)
QM9_train,QM9_val,QM9_test=split_list(QM9,(0.8,0.1,0.1))

In [13]:
from torch_geometric.loader import DenseDataLoader,DataLoader


train_loader=DataLoader(QM9_train,batch_size=128, shuffle=True)
val_loader = DataLoader(QM9_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(QM9_test, batch_size=1024, shuffle=False)
num_features=QM9_train[0].num_node_features

In [14]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.039286    Val MAE:0.019826    Test MAE:0.019966     Time:11.963722 s
Epoch 2
Train MAE:0.019430    Val MAE:0.019534    Test MAE:0.019609     Time:13.157035 s
Epoch 3
Train MAE:0.018271    Val MAE:0.017862    Test MAE:0.017881     Time:12.383650 s
Epoch 4
Train MAE:0.017506    Val MAE:0.017703    Test MAE:0.017744     Time:12.343323 s
Epoch 5
Train MAE:0.017043    Val MAE:0.018043    Test MAE:0.018088     Time:12.704738 s
Epoch 6
Train MAE:0.016426    Val MAE:0.015581    Test MAE:0.015679     Time:12.655563 s
Epoch 7
Train MAE:0.015947    Val MAE:0.015239    Test MAE:0.015315     Time:12.647420 s
Epoch 8
Train MAE:0.015491    Val MAE:0.014903    Test MAE:0.015022     Time:12.630172 s
Epoch 9
Train MAE:0.015019    Val MAE:0.015480    Test MAE:0.015506     Time:12.707192 s
Epoch 10
Train MAE:0.014719    Val MAE:0.013871    Test MAE:0.014002     Time:12.859712 s
Epoch 11
Train MAE:0.014375    Val MAE:0.013869    Test MAE:0.014009     Time:12.746305 s
Epoch 12
Train MAE:

In [15]:
epochs=50
train_loader=DataLoader(QM9_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.009975    Val MAE:0.010627    Test MAE:0.010705     Time:27.185938 s
Epoch 2
Train MAE:0.009942    Val MAE:0.010039    Test MAE:0.010110     Time:27.159919 s
Epoch 3
Train MAE:0.009935    Val MAE:0.010399    Test MAE:0.010438     Time:27.478165 s
Epoch 4
Train MAE:0.009868    Val MAE:0.010110    Test MAE:0.010159     Time:26.498406 s
Epoch 5
Train MAE:0.009857    Val MAE:0.009486    Test MAE:0.009543     Time:26.296062 s
Epoch 6
Train MAE:0.009891    Val MAE:0.009690    Test MAE:0.009739     Time:26.623340 s
Epoch 7
Train MAE:0.009813    Val MAE:0.009353    Test MAE:0.009425     Time:26.623347 s
Epoch 8
Train MAE:0.009725    Val MAE:0.009958    Test MAE:0.009969     Time:26.410673 s
Epoch 9
Train MAE:0.009776    Val MAE:0.009306    Test MAE:0.009412     Time:26.342700 s
Epoch 10
Train MAE:0.009692    Val MAE:0.010033    Test MAE:0.010125     Time:28.227495 s
Epoch 11
Train MAE:0.009599    Val MAE:0.009196    Test MAE:0.009248     Time:28.447750 s
Epoch 12
Train MAE:

In [16]:
with open("./result/RING_QM9_3.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))

# OPV

In [5]:
from torch_geometric.seed import seed_everything


max_nodes=122
OPV=load_dataset("./dataset/OPV/OPV.csv","smile","homo","lumo")

100%|████████████████████████████████████████████████████████████████████████████| 90823/90823 [17:22<00:00, 87.10it/s]


### Exp 1

In [7]:
seed_everything(1222)
OPV_train,OPV_val,OPV_test=split_list(OPV,(0.8,0.1,0.1))

In [8]:
from torch_geometric.loader import DenseDataLoader,DataLoader

train_loader=DataLoader(OPV_train,batch_size=128, shuffle=True)
val_loader = DataLoader(OPV_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(OPV_test, batch_size=1024, shuffle=False)
num_features=OPV_train[0].num_node_features

In [9]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.463740    Val MAE:0.272906    Test MAE:0.273274     Time:14.623544 s
Epoch 2
Train MAE:0.268637    Val MAE:0.260347    Test MAE:0.260564     Time:13.861656 s
Epoch 3
Train MAE:0.254833    Val MAE:0.238230    Test MAE:0.239736     Time:13.922180 s
Epoch 4
Train MAE:0.244952    Val MAE:0.248867    Test MAE:0.248675     Time:14.302477 s
Epoch 5
Train MAE:0.240523    Val MAE:0.234533    Test MAE:0.236960     Time:14.420443 s
Epoch 6
Train MAE:0.233552    Val MAE:0.222829    Test MAE:0.224555     Time:14.658813 s
Epoch 7
Train MAE:0.228031    Val MAE:0.212818    Test MAE:0.214743     Time:14.868886 s
Epoch 8
Train MAE:0.222601    Val MAE:0.212670    Test MAE:0.212905     Time:15.051206 s
Epoch 9
Train MAE:0.219167    Val MAE:0.203873    Test MAE:0.204314     Time:14.926129 s
Epoch 10
Train MAE:0.215982    Val MAE:0.226949    Test MAE:0.227929     Time:15.096057 s
Epoch 11
Train MAE:0.213571    Val MAE:0.198211    Test MAE:0.198082     Time:15.531018 s
Epoch 12
Train MAE:

In [10]:
epochs=50
train_loader=DataLoader(OPV_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.168288    Val MAE:0.159878    Test MAE:0.159696     Time:27.079040 s
Epoch 2
Train MAE:0.167340    Val MAE:0.171106    Test MAE:0.170451     Time:27.863184 s
Epoch 3
Train MAE:0.167158    Val MAE:0.194168    Test MAE:0.194500     Time:29.290803 s
Epoch 4
Train MAE:0.165629    Val MAE:0.164083    Test MAE:0.163156     Time:30.185520 s
Epoch 5
Train MAE:0.163798    Val MAE:0.169360    Test MAE:0.168595     Time:30.559797 s
Epoch 6
Train MAE:0.164453    Val MAE:0.156782    Test MAE:0.157076     Time:30.840491 s
Epoch 7
Train MAE:0.163372    Val MAE:0.157464    Test MAE:0.156279     Time:31.340067 s
Epoch 8
Train MAE:0.162661    Val MAE:0.159185    Test MAE:0.158306     Time:31.450239 s
Epoch 9
Train MAE:0.162580    Val MAE:0.158789    Test MAE:0.158027     Time:31.622804 s
Epoch 10
Train MAE:0.162184    Val MAE:0.158976    Test MAE:0.157627     Time:32.941289 s
Epoch 11
Train MAE:0.161591    Val MAE:0.158913    Test MAE:0.159282     Time:32.142270 s
Epoch 12
Train MAE:

In [11]:
with open("./result/RING_OPV_1.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))

### Exp 2

In [12]:
seed_everything(324)
OPV_train,OPV_val,OPV_test=split_list(OPV,(0.8,0.1,0.1))

In [13]:
from torch_geometric.loader import DenseDataLoader,DataLoader

train_loader=DataLoader(OPV_train,batch_size=128, shuffle=True)
val_loader = DataLoader(OPV_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(OPV_test, batch_size=1024, shuffle=False)
num_features=OPV_train[0].num_node_features

In [14]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.429022    Val MAE:0.300691    Test MAE:0.301553     Time:13.447194 s
Epoch 2
Train MAE:0.288521    Val MAE:0.319651    Test MAE:0.321415     Time:13.913639 s
Epoch 3
Train MAE:0.276111    Val MAE:0.274477    Test MAE:0.276899     Time:14.488191 s
Epoch 4
Train MAE:0.265000    Val MAE:0.251078    Test MAE:0.252436     Time:14.391657 s
Epoch 5
Train MAE:0.257828    Val MAE:0.253336    Test MAE:0.255456     Time:14.358721 s
Epoch 6
Train MAE:0.247473    Val MAE:0.250327    Test MAE:0.251652     Time:14.676046 s
Epoch 7
Train MAE:0.243836    Val MAE:0.267534    Test MAE:0.267811     Time:14.811954 s
Epoch 8
Train MAE:0.238826    Val MAE:0.225351    Test MAE:0.226539     Time:15.077104 s
Epoch 9
Train MAE:0.231091    Val MAE:0.218678    Test MAE:0.219795     Time:15.164153 s
Epoch 10
Train MAE:0.228425    Val MAE:0.227664    Test MAE:0.228564     Time:15.462903 s
Epoch 11
Train MAE:0.223684    Val MAE:0.238134    Test MAE:0.240104     Time:16.733869 s
Epoch 12
Train MAE:

In [15]:
epochs=50
train_loader=DataLoader(OPV_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.168312    Val MAE:0.164667    Test MAE:0.164851     Time:33.878551 s
Epoch 2
Train MAE:0.166633    Val MAE:0.168454    Test MAE:0.168030     Time:33.366809 s
Epoch 3
Train MAE:0.166536    Val MAE:0.157458    Test MAE:0.157854     Time:32.655408 s
Epoch 4
Train MAE:0.164688    Val MAE:0.174030    Test MAE:0.173670     Time:32.770563 s
Epoch 5
Train MAE:0.164500    Val MAE:0.159631    Test MAE:0.159778     Time:33.474158 s
Epoch 6
Train MAE:0.162887    Val MAE:0.184623    Test MAE:0.184102     Time:35.108790 s
Epoch 7
Train MAE:0.161680    Val MAE:0.165758    Test MAE:0.165332     Time:34.022787 s
Epoch 8
Train MAE:0.161807    Val MAE:0.158806    Test MAE:0.158758     Time:34.750014 s
Epoch 9
Train MAE:0.161423    Val MAE:0.153813    Test MAE:0.154133     Time:34.189102 s
Epoch 10
Train MAE:0.158976    Val MAE:0.161532    Test MAE:0.162012     Time:34.751580 s
Epoch 11
Train MAE:0.159636    Val MAE:0.160561    Test MAE:0.160199     Time:34.672621 s
Epoch 12
Train MAE:

In [16]:
with open("./result/RING_OPV_2.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))

### Exp 3

In [17]:
seed_everything(10086)
OPV_train,OPV_val,OPV_test=split_list(OPV,(0.8,0.1,0.1))

In [18]:
from torch_geometric.loader import DenseDataLoader,DataLoader

train_loader=DataLoader(OPV_train,batch_size=128, shuffle=True)
val_loader = DataLoader(OPV_val, batch_size=1024, shuffle=False)
test_loader = DataLoader(OPV_test, batch_size=1024, shuffle=False)
num_features=OPV_train[0].num_node_features

In [19]:
from torch.nn import MSELoss
import time

ringpool=RingPool(num_features)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.001)
loss_fn=MSELoss()

ringpool=ringpool.cuda()
loss_fn=loss_fn.cuda()

epochs=50
train_curve=[]
val_curve=[]
test_curve=[]
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.615568    Val MAE:0.298563    Test MAE:0.299125     Time:13.519033 s
Epoch 2
Train MAE:0.286755    Val MAE:0.269729    Test MAE:0.269793     Time:14.024954 s
Epoch 3
Train MAE:0.270946    Val MAE:0.288959    Test MAE:0.289749     Time:14.146966 s
Epoch 4
Train MAE:0.259993    Val MAE:0.260959    Test MAE:0.260486     Time:14.565308 s
Epoch 5
Train MAE:0.245817    Val MAE:0.237576    Test MAE:0.238443     Time:14.909628 s
Epoch 6
Train MAE:0.234588    Val MAE:0.230017    Test MAE:0.231555     Time:15.888103 s
Epoch 7
Train MAE:0.224737    Val MAE:0.218382    Test MAE:0.219459     Time:16.095260 s
Epoch 8
Train MAE:0.220774    Val MAE:0.240064    Test MAE:0.241218     Time:15.929678 s
Epoch 9
Train MAE:0.218673    Val MAE:0.210337    Test MAE:0.211236     Time:16.161938 s
Epoch 10
Train MAE:0.216538    Val MAE:0.213273    Test MAE:0.214093     Time:16.089619 s
Epoch 11
Train MAE:0.213102    Val MAE:0.213272    Test MAE:0.215014     Time:16.490868 s
Epoch 12
Train MAE:

In [20]:
epochs=50
train_loader=DataLoader(OPV_train,batch_size=32, shuffle=True)
optimizer=torch.optim.Adam(params=ringpool.parameters(), lr=0.0005)
for i in range(1,epochs+1):
    t_start=time.time()
    ringpool,train_mae = train(ringpool, train_loader, optimizer, loss_fn)
    val_mae=test(ringpool,val_loader)
    test_mae=test(ringpool,test_loader)
    t_end=time.time()
    print("Epoch %d"%i)
    print("Train MAE:%f    Val MAE:%f    Test MAE:%f     Time:%f s"%(train_mae,val_mae,test_mae,t_end-t_start))
    
    train_curve.append(train_mae)
    val_curve.append(val_mae)
    test_curve.append(test_mae)

Epoch 1
Train MAE:0.172753    Val MAE:0.184926    Test MAE:0.185749     Time:28.345949 s
Epoch 2
Train MAE:0.171670    Val MAE:0.165159    Test MAE:0.165924     Time:29.357146 s
Epoch 3
Train MAE:0.170395    Val MAE:0.169529    Test MAE:0.169977     Time:31.037376 s
Epoch 4
Train MAE:0.168680    Val MAE:0.160409    Test MAE:0.161362     Time:31.696175 s
Epoch 5
Train MAE:0.168975    Val MAE:0.166046    Test MAE:0.166636     Time:31.062864 s
Epoch 6
Train MAE:0.168868    Val MAE:0.168323    Test MAE:0.168680     Time:31.506913 s
Epoch 7
Train MAE:0.167076    Val MAE:0.192197    Test MAE:0.194540     Time:33.320707 s
Epoch 8
Train MAE:0.167173    Val MAE:0.180786    Test MAE:0.182133     Time:36.029058 s
Epoch 9
Train MAE:0.165743    Val MAE:0.164239    Test MAE:0.165466     Time:34.717663 s
Epoch 10
Train MAE:0.166300    Val MAE:0.161557    Test MAE:0.162387     Time:33.749334 s
Epoch 11
Train MAE:0.165078    Val MAE:0.162651    Test MAE:0.163451     Time:32.722123 s
Epoch 12
Train MAE:

In [21]:
with open("./result/RING_OPV_3.csv","w") as fo:
    fo.write("epoch,train,val,test\n")
    for i in range(len(train_curve)):
        fo.write("%d,%f,%f,%f\n"%(i+1,train_curve[i],val_curve[i],test_curve[i]))